In [2]:
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf

import matplotlib.pyplot as plt

from tensorflow import keras
from sklearn.model_selection import StratifiedShuffleSplit

# Understanding Data

Memahami data ini bertujuan untuk mendapatkan insight-insight baru terhadap data.

In [3]:
anime_df = pd.read_csv("../input/anime-recommendations-database/anime.csv")
rating_df = pd.read_csv("../input/anime-recommendations-database/rating.csv")

## Deskripsi Data

Pada file Anime.csv, terdapat variabel-variabel sebagai berikut:

- anime_id : Merupakan identifier unik film anime.
- name : Merupakan judul lengkap anime.
- genre : Merupakan genre anime.
- type : Merupakan jenis anime (OVA, TV, Movie).
- episodes : Merupakan jumlah episode anime tersebut.
- rating : Merupakan rerata nilai yang didapatkan anime tersebut.
- members : Jumlah anggota yang berada pada "grup" anime tersebut.

In [3]:
anime_df.head(10)

Pada file Rating.csv, terdapat variabel-variabel sebagai berikut:

- user_id : Merupakan identifier yang dibuat secara random sebagai user_id.
- anime_id : Merupakan identifier anime yang diberikan nilai oleh pengguna.
- rating : Merupakan nilai yang diberikan oleh pengguna, memiliki skala hingga 10 serta nilai -1 jika user menonton dan tidak memberikan rating.

In [4]:
rating_df.head(10)

## Memeriksa Jumlah Data

Berdasarkan hipotesis solusi yang diajukan menggunakan pendekatan *Collaborative Filtering* maka fitur-fitur yang akan diperiksa adalah sebagai berikut.
1. Jumlah seluruh data penilaian terhadap *anime*
2. Jumlah user identifier
3. Jumlah anime identifier

In [4]:
print("Jumlah seluruh data penilaian anime: {}".format(len(rating_df["user_id"])))
print("Jumlah identifier unik user: {}".format(len(rating_df["user_id"].unique())))
print("Jumlah identifier unik anime: {}".format(len(anime_df["anime_id"].unique())))

## Memeriksa Keadaan Data

Memeriksa keadaan dataset apakah terdapat *Missing Value*.

In [6]:
anime_df.info()

In [7]:
rating_df.info()

In [8]:
rating_df.isnull().sum()

# Data Cleaning

## Mengatasi Anime yang Tidak Diberi Nilai

Berdasarkan metadata singkat dataset, terdapat rating -1 yang berarti pengguna menonton *anime* tersebut namun tidak menilai *anime* tersebut. Hal ini akan menjadi masalah, karena kita tidak mengetahui apakah pengguna tersebut menyukai *anime* tersebut. 

In [9]:
len(rating_df[rating_df["rating"] == -1])

In [10]:
len(rating_df[rating_df["rating"] == -1]) / len(rating_df) * 100

In [5]:
neg_rating = rating_df[rating_df["rating"] == -1]
pos_rating = rating_df[rating_df["rating"] != -1]

print("Jumlah data yang memiliki poitif rating: {}".format(len(pos_rating)))
print("Jumlah data yang memiliki negatif rating: {}".format(len(neg_rating)))
print("Jumlah identifier unik pada data negatif rating: {}".format(len(neg_rating["anime_id"].unique())))

In [12]:
neg_rating.head(5)

In [6]:
merge_rating = neg_rating.merge(anime_df, on="anime_id")

merge_rating = merge_rating[["user_id", "anime_id", "rating_y"]]
print("Panjang sebelum menghilangkan missing value: {}".format(len(merge_rating["anime_id"].unique())))

merge_rating = merge_rating.dropna(subset=["rating_y"])
print("Panjang setelah menghilangkan missing value: {}".format(len(merge_rating["anime_id"].unique())))

merge_rating["rating_y"] = merge_rating["rating_y"].astype("int64")
print("Merubah tipe data rating_y dari float menjadi int!")

In [14]:
merge_rating.info()

In [7]:
merge_rating = merge_rating.rename(columns={"rating_y":"rating"})

In [8]:
prepared_df = pd.concat([pos_rating, merge_rating], ignore_index=True)

print("Jumlah data setelah dilakukan penggabungan data {}".format(len(prepared_df)))

In [17]:
prepared_df.head(10)

# Eksplorasi Data: Univariate Analysis

Memvisualisasikan ketersebaran rating terhadap jumlah data yang ada.

### Melihat Ketersebaran Data Rating

In [18]:
prepared_df.groupby("rating").size().plot(kind="bar", title="Distribution of Rating")

In [19]:
prepared_df["rating"].value_counts() / len(prepared_df) * 100

### Melihat Jenis Anime Terpopuler

In [20]:
anime_df["type"].value_counts().plot(kind="bar", title="Popular Anime Types")

# Data Preparation

Mempersiapkan data dimulai dari:
1. Pembuatan dictionary encoded baru
2. Split data menjadi training dan testing

### Encode User_id dan Anime_id

Meng-encode fitur user_id dan anime_id menjadi indeks integer. Hal ini dilakukan karena dua hal:
1. Sebagai look up dictionary
2. Indeks row embedding nantinya akan mengikuti indeks ini

In [9]:
encoded_df = prepared_df.copy()

Meng-encode menggunakan metode pandas .cat dan .codes. Kolom yang akan di-return indeks dari kolom tersebut.

In [10]:
encoded_df["user_id"] = encoded_df["user_id"].astype("category") 
encoded_df["user_id_encoded"] = encoded_df["user_id"].cat.codes

encoded_df["anime_id"] = encoded_df["anime_id"].astype("category")
encoded_df["anime_id_encoded"] = encoded_df["anime_id"].cat.codes

In [11]:
print("Jumlah user_id: {}".format(len(encoded_df["user_id"].unique())))
print("Jumlah hasil encoded user_id: {}".format(len(encoded_df["user_id_encoded"].unique())))
print()
print("Jumlah anime_id: {}".format(len(encoded_df["anime_id"].unique())))
print("Jumlah hasil encoded anime_id: {}".format(len(encoded_df["anime_id_encoded"].unique())))

In [24]:
encoded_df.sort_values("user_id_encoded").head(10)

In [25]:
encoded_df.sort_values("anime_id_encoded").head(10)

### Membangun dictionary sebagai lookup table

In [27]:
user_id_decoder = dict(zip(encoded_df["user_id_encoded"], encoded_df["user_id"]))
anime_id_decoder = dict(zip(encoded_df["anime_id_encoded"], encoded_df["anime_id"]))

user_id_encoder = dict(zip(encoded_df["user_id"], encoded_df["user_id_encoded"]))
anime_id_encoder = dict(zip(encoded_df["anime_id"], encoded_df["anime_id_encoded"]))

### Splitting Data

Splitting data dengan membagi data sesuai representasi keseluruhan data

In [13]:
encoded_prep_df = encoded_df[["user_id_encoded", "anime_id_encoded", "rating"]]

split = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=42)
for train_index, test_index in split.split(encoded_prep_df, encoded_prep_df["rating"]):
    strat_train_set = encoded_prep_df.loc[train_index]
    strat_test_set = encoded_prep_df.loc[test_index]

Hasil pembagian data secara strata

In [28]:
strat_train_set.groupby("rating").size() / len(strat_train_set) * 100

In [29]:
strat_test_set.groupby("rating").size() / len(strat_test_set) * 100

In [14]:
X_train = strat_train_set[["user_id_encoded", "anime_id_encoded"]].to_numpy()
y_train = strat_train_set["rating"].to_numpy()
y_train = (y_train - min(y_train)) / (max(y_train) - min(y_train))

X_test = strat_test_set[["user_id_encoded", "anime_id_encoded"]].to_numpy()
y_test = strat_test_set["rating"].to_numpy()
y_test = (y_test - min(y_test)) / (max(y_test) - min(y_test))

### Hasil Data Preparation dalam bentuk Numpy

In [15]:
print(X_train, y_train)

# Modelling Data

Pada bagian ini akan dilakukan modelling menggunakan pendekatan Deep Learning:
1. RecommenderNet
2. Probabilistic Matrix Factorization

In [16]:
## Global Variable

NUM_USERS = len(user_id_dict)
NUM_ITEMS = len(anime_id_dict)
EMBEDDING_SIZE = 100

## RecommenderNet

In [17]:
class RecommenderNet(tf.keras.Model):
    def __init__(self, n_users, n_items, embedding_size, **kwargs):
        super(RecommenderNet, self).__init__(**kwargs)
        self.n_users = n_users
        self.n_items = n_items
        self.embedding_size = embedding_size
        
        self.user_embedding = keras.layers.Embedding(n_users,
                                                     embedding_size,
                                                     embeddings_initializer = 'he_normal',
                                                     embeddings_regularizer = keras.regularizers.l2(1e-6))
        
        self.user_bias = keras.layers.Embedding(n_users, 1)
        
        self.items_embedding = keras.layers.Embedding(n_items,
                                                      embedding_size,
                                                      embeddings_initializer = 'he_normal',
                                                      embeddings_regularizer = keras.regularizers.l2(1e-6))
        self.items_bias = keras.layers.Embedding(n_items, 1)
    
    def call(self, inputs):
        user_vector = self.user_embedding(inputs[:,0])
        user_bias = self.user_bias(inputs[:, 0])
        item_vector = self.items_embedding(inputs[:, 1])
        item_bias = self.items_bias(inputs[:, 1])

        dot_user_resto = tf.tensordot(user_vector, item_vector, 2) 

        x = dot_user_resto + user_bias + item_bias

        return tf.nn.sigmoid(x)

In [19]:
recommender_net = RecommenderNet(NUM_USERS, NUM_ITEMS, EMBEDDING_SIZE)

recommender_net.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                        metrics=[tf.keras.metrics.RootMeanSquaredError()])

In [20]:
recommender_net_hist = recommender_net.fit(X_train, y_train,
                                           epochs=1,
                                           validation_data=(X_test, y_test))

## Probabilistic Matrix Factorization

In [33]:
class PMF(tf.keras.Model):
    def __init__(self, n_users, n_items, n_dim):
        super(PMF, self).__init__()      
        self.w_u_i_init = tf.keras.initializers.RandomUniform(minval=-1., maxval=1., seed=1)
        
        self.user_embedding = tf.keras.layers.Embedding(n_users,
                                                        n_dim,
                                                        embeddings_initializer='uniform',
                                                        embeddings_regularizer=tf.keras.regularizers.L2(0.1))
      
        self.item_embedding = tf.keras.layers.Embedding(n_items,
                                                        n_dim,
                                                        embeddings_initializer='uniform',
                                                        embeddings_regularizer=tf.keras.regularizers.L2(0.1))
        
        ## users embedding
        self.ub = tf.keras.layers.Embedding(n_users, 
                                            1, 
                                            embeddings_initializer=self.w_u_i_init, 
                                            embeddings_regularizer=tf.keras.regularizers.L2(0.1)) 
        ## items embedding
        self.ib = tf.keras.layers.Embedding(n_items, 
                                            1, 
                                            embeddings_initializer=self.w_u_i_init, 
                                            embeddings_regularizer=tf.keras.regularizers.L2(0.1))
        
    def call(self, inputs):
        self.user_index = inputs[:, 0]
        self.item_index = inputs[:, 1]
        
        user_h1 = self.user_embedding(self.user_index)
        item_h1 = self.item_embedding(self.item_index)
        
        r_h = tf.math.reduce_sum(user_h1 * item_h1, axis=1 if len(user_h1.shape) > 1 else 0)
        r_h += tf.squeeze(self.ub(self.user_index))
        r_h += tf.squeeze(self.ib(self.item_index))
        
        return tf.nn.sigmoid(r_h)

In [40]:
keras.backend.clear_session()

pmf = PMF(NUM_USERS, NUM_ITEMS, EMBEDDING_SIZE)

pmf.compile(loss=tf.keras.losses.BinaryCrossentropy(),
            optimizer='adam',
            metrics=[tf.keras.metrics.RootMeanSquaredError()])

In [41]:
pmf_hist = pmf.fit(X_train, y_train,
                   epochs=5,
                   validation_data=(X_test, y_test))

# Evaluasi Model

In [44]:
def plot_model(history):
    plt.plot(history.history['root_mean_squared_error'])
    plt.plot(history.history['val_root_mean_squared_error'])
    plt.title('model_metrics')
    plt.ylabel('root_mean_squared_error')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

In [45]:
plot_model(pmf_hist)

In [49]:
plot_model(recommender_net_hist)

# Recommendation Result

In [35]:
user_id = rating_df["user_id"].sample(1).iloc[0]
anime_watched_by_user = rating_df[rating_df["user_id"] == user_id]
anime_not_watched = rating_df[~rating_df["anime_id"].isin(anime_watched_by_user["anime_id"].values)]["anime_id"]
anime_not_watched = list(
    set(anime_not_watched).intersection(set(anime_id_encoder.keys()))
)
anime_not_watched = [[anime_id_encoder.get(x)] for x in anime_not_watched]
user_encoder = user_id_encoder.get(user_id)
user_anime_array = np.hstack(
    ([[user_encoder]] * len(anime_not_watched), anime_not_watched)
)

In [46]:
ratings = recommender_net.predict(user_anime_array).flatten()
top_ratings_indices = ratings.argsort()[-10:][::-1]
recommended_anime_ids = [
    anime_id_decoder.get(anime_not_watched[x][0]) for x in top_ratings_indices
]

In [48]:
print("Showing recommendations for user: {}".format(user_id))
print("====" * 9)
print("Movies with high ratings from user")
print("----" * 8)
top_anime_user = (
    anime_watched_by_user.sort_values(by="rating", ascending=False)
    .head(5)
    .anime_id.values
)
anime_df_rows = anime_df[anime_df["anime_id"].isin(top_anime_user)]
for row in anime_df_rows.itertuples():
    print(row.name, ":", row.genre)

print("----" * 8)
print("Top 10 movie recommendations")
print("----" * 8)
recommended_anime = anime_df[anime_df["anime_id"].isin(recommended_anime_ids)]
for row in recommended_anime.itertuples():
    print(row.name, ":", row.genre)